Packages:

In [42]:
import glob
import os
import warnings
 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom
from pydicom.data import get_testdata_files
from sklearn.model_selection import train_test_split

from IPython.display import Markdown, display

 # Import functions from the module
import importlib
import help_files._0_definitions 
import  help_files._1_visuals_script
# import  help_files._01_load_data
 # Reload the module to apply the changes to the script
importlib.reload(help_files._0_definitions)
importlib.reload(help_files._1_visuals_script)
# importlib.reload(help_files._01_load_data)
import  help_files._1_visuals_script  as pauls_vs
# Group by 'condition', 'level', and 'severity' and count occurrences
from help_files._0_definitions import count_severity_by_condition_level 
# Define the path
from pathlib import Path

pd.set_option("display.width", 1000)  # Set a large width to prevent line wrapping
 

In [43]:
### In definitions are all the functions that are used in the notebook and globals
with open("help_files/_0_definitions.py") as file:
    exec(file.read())



In [ ]:
 

# Specify the file names and DataFrame variable names
file_names = ["test_df_3_cat.csv"]

# Load the data from the CSV files
dataframes = [pd.read_csv(data_path_vor / file_name) for file_name in file_names]
# Load train_df as a DataFrame
test_df = dataframes[0]
test_df = test_df.sample(frac=0.5, random_state=RSEED)
print("DataFrames have been loaded successfully.")

Restore parameters and metrtics from mlflow

I should take prdicted probabilities and not predicted classes: output scores (logits) are converted into probabilities using the softmax function

In [45]:
import mlflow
import mlflow.pytorch
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.models as models
from torchvision import transforms
import pydicom
import cv2
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

# Set the experiment name
experiment_name = "407167111170960490"
mlflow.set_experiment(experiment_name)

# Specify the run ID of the logged model
run_id = "e2279f9e4abb43f9824a1ba402ce8168"  # Replace with your actual run ID
model_uri = f"runs:/{run_id}/model"

# Load the model
model = mlflow.pytorch.load_model(model_uri)
model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))  # Move model to appropriate device
model.eval()  # Set the model to evaluation mode

# Prepare data
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class MRIDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        self.data['severity'] = self.data['severity'].astype(int)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        image_path = row['image_path']
        label = row['severity']
        dicom_image = pydicom.dcmread(image_path)
        image = dicom_image.pixel_array.astype(float)
        image = (image / image.max() * 255).astype('uint8')
        if len(image.shape) == 2:  # Grayscale
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        image_tensor = self.transform(image) if self.transform else torch.from_numpy(image).permute(2, 0, 1)
        return image_tensor, torch.tensor(label).long()

    def __len__(self):
        return len(self.data)

predicting probabilities for test data

In [46]:
test_data = test_df

In [47]:
test_dataset = MRIDataset(data=test_data, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Inference loop with probability extraction
results = []
probabilities_list = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)  # Ensure labels are also on the correct device
        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1)  # Calculate class probabilities
        _, predicted_classes = torch.max(outputs, 1)
        
        # Append predictions and probabilities
        results.append(predicted_classes.item())
        probabilities_list.append(probabilities.cpu().numpy())



In [48]:
# Print predicted classes and their probabilities
for i, (pred, probs) in enumerate(zip(results, probabilities_list)):
    print(f"Test image {i}: Predicted class {pred}, Probabilities: {probs}")

# Additional code to plot the confusion matrix can stay as-is:
true_labels = []
predicted_labels = []

for images, labels in test_loader:
    images = images.to(device)
    true_labels.extend(labels.numpy())

    with torch.no_grad():
        outputs = model(images)
        _, predicted_classes = torch.max(outputs, 1)
        predicted_labels.extend(predicted_classes.cpu().numpy())

true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

print("True Labels Unique Values:", np.unique(true_labels))
print("Predicted Labels Unique Values:", np.unique(predicted_labels))


Test image 0: Predicted class 1, Probabilities: [[0.00468007 0.67013323 0.3251867 ]]
Test image 1: Predicted class 2, Probabilities: [[1.59805131e-04 1.53880585e-02 9.84452128e-01]]
Test image 2: Predicted class 1, Probabilities: [[8.4838719e-04 9.9664080e-01 2.5107723e-03]]
Test image 3: Predicted class 0, Probabilities: [[0.9588287  0.0389594  0.00221194]]
Test image 4: Predicted class 1, Probabilities: [[0.447508   0.5506946  0.00179744]]
Test image 5: Predicted class 0, Probabilities: [[9.7433633e-01 2.5502523e-02 1.6115354e-04]]
Test image 6: Predicted class 1, Probabilities: [[1.3570620e-03 9.9864095e-01 1.9822594e-06]]
Test image 7: Predicted class 0, Probabilities: [[0.8226465  0.1762069  0.00114661]]
Test image 8: Predicted class 1, Probabilities: [[3.6824416e-04 9.9912840e-01 5.0338294e-04]]
Test image 9: Predicted class 1, Probabilities: [[7.9903041e-04 9.9778086e-01 1.4200711e-03]]
Test image 10: Predicted class 0, Probabilities: [[9.908626e-01 9.045723e-03 9.168630e-05]]
T

In [49]:
# Convert the list of probabilities to a numpy array for easier manipulation
probabilities_array = np.vstack(probabilities_list)

# Ensure the length of probabilities_array matches the length of test_df
probabilities_array = probabilities_array[:len(test_df)]

# Add the probabilities to the test_df DataFrame
test_df['Probability_Class_0'] = probabilities_array[:, 0]
test_df['Probability_Class_1'] = probabilities_array[:, 1]
test_df['Probability_Class_2'] = probabilities_array[:, 2]

# Round the probabilities to 4 decimal places
test_df['Probability_Class_0'] = test_df['Probability_Class_0'].round(4)
test_df['Probability_Class_1'] = test_df['Probability_Class_1'].round(4)
test_df['Probability_Class_2'] = test_df['Probability_Class_2'].round(4)
# Add the predicted classes to the test_df DataFrame
test_df['Predicted_Class'] = results



# Display the updated DataFrame
 
test_df.head()

,study_id,severity,condition,level,series_id,x,y,image_path,missing_image,Probability_Class_0,Probability_Class_1,Probability_Class_2,Predicted_Class
6296,2775207739,1,0,l5/s1,3.249541e+09,157.538462,250.197802,data/train_images_origin/2775207739/3249541180...,False,0.0047,0.6701,0.3252,1
1496,664153360,2,0,l5/s1,1.076246e+09,260.194392,306.003738,data/train_images_origin/664153360/1076245514/...,False,0.0002,0.0154,0.9845,2
5040,2273432465,1,0,l5/s1,1.257235e+09,234.036660,287.804481,data/train_images_origin/2273432465/114306451/...,False,0.0008,0.9966,0.0025,1
8486,3777149998,0,0,l5/s1,7.355834e+08,151.528158,190.928463,data/train_images_origin/3777149998/3550597941...,False,0.9588,0.0390,0.0022,0
8959,3967802493,0,0,l5/s1,1.589249e+09,265.649924,385.753425,data/train_images_origin/3967802493/2054070341...,False,0.4475,0.5507,0.0018,1


In [50]:
# Group by severity and sum the Probability_Class_1
severity_prob_sum_all_classes = test_df.groupby('severity')[['Probability_Class_0', 'Probability_Class_1', 'Probability_Class_2']].mean()

# Print the result
severity_prob_sum_all_classes
# Use severity_prob_sum_all_classes as confusion matrix
conf_matrix_prob_df = severity_prob_sum_all_classes 
# Create a confusion matrix from severity and predicted class
conf_matrix_severity_pred = confusion_matrix(test_df['severity'], test_df['Predicted_Class'])

# Create a DataFrame from the confusion matrix
conf_matrix_severity_pred_df = pd.DataFrame(
    conf_matrix_severity_pred, 
    index=[f"Actual {i}" for i in range(len(conf_matrix_severity_pred))], 
    columns=[f"Predicted {i}" for i in range(len(conf_matrix_severity_pred[0]))]
)

# Print the confusion matrix DataFrame
print(conf_matrix_severity_pred_df)

# Optionally, display it using a more formatted view (e.g., in Jupyter Notebook)
conf_matrix_severity_pred_df.style.background_gradient(cmap='Blues')
# Print the confusion matrix DataFrame
print(conf_matrix_prob_df)

# Optionally, display it using a more formatted view (e.g., in Jupyter Notebook)
conf_matrix_prob_df.style.background_gradient(cmap='Blues')


          Predicted 0  Predicted 1  Predicted 2
Actual 0         1282          280           12
Actual 1           29         1594           14
Actual 2           12           83         1480
          Probability_Class_0  Probability_Class_1  Probability_Class_2
severity                                                               
0                    0.754056             0.230405             0.015539
1                    0.034404             0.947798             0.017795
2                    0.015796             0.077630             0.906572


,Probability_Class_0,Probability_Class_1,Probability_Class_2
severity,,,
0,0.754056,0.230405,0.015539
1,0.034404,0.947798,0.017795
2,0.015796,0.077630,0.906572


In [51]:
conf_matrix_severity_pred_df.style.background_gradient(cmap='Blues')

,Predicted 0,Predicted 1,Predicted 2
Actual 0,1282,280,12
Actual 1,29,1594,14
Actual 2,12,83,1480


In [52]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np

# Ensure true_labels and predicted_labels have the same length
min_length = min(len(true_labels), len(predicted_labels))
true_labels = true_labels[:min_length]
predicted_labels = predicted_labels[:min_length]

# Generate the confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Create a DataFrame from the confusion matrix
conf_matrix_df = pd.DataFrame(
    conf_matrix, 
    index=[f"Actual {i}" for i in range(len(conf_matrix))], 
    columns=[f"Predicted {i}" for i in range(len(conf_matrix[0]))]
)

# Print the confusion matrix DataFrame
print(conf_matrix_df)

# Optionally, display it using a more formatted view (e.g., in Jupyter Notebook)
conf_matrix_df.style.background_gradient(cmap='Blues')


          Predicted 0  Predicted 1  Predicted 2
Actual 0         1286          275           13
Actual 1           28         1595           14
Actual 2           15           93         1467


,Predicted 0,Predicted 1,Predicted 2
Actual 0,1286,275,13
Actual 1,28,1595,14
Actual 2,15,93,1467


In [53]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
# Calculate precision, recall, and accuracy using mean_probabilities_df
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
accuracy = accuracy_score(true_labels, predicted_labels)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")


Precision: 0.9193
Recall: 0.9085
Accuracy: 0.9085
